Covid-19 Dashboard Kit (C) Daniella Kavanna, 2021 (daniellakavanna@gmail.com). All rights reserved.

#  Covid-19 Dashboard

A Covid-19 Dashboard has been created to visualise a portion of the influx of data initiated by the current pandemic. The graph below shows a comparison of two metrics; the cumulative vaccination percentage against the cumulative deaths in the region London. 

Click to refresh data from Public Health England PHE. 



In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("data_new.json", "rt") as INFILE:
    jsondata=json.load(INFILE)

In [18]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")



def wrangle_data(rawdata):
    
    datalist=jsondata['data']
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    data_newdf=pd.DataFrame(index=index, columns=['vaccination_coverage', 'deaths'])
    
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['vaccination_coverage', 'deaths']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(data_newdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                data_newdf.loc[date, column]=value
            
    # fill in any remaining "holes" due to missing dates
    data_newdf.fillna(0.0, inplace=True)
            
    return data_newdf
    

            
data_newdf=wrangle_data(jsondata)
            

In [20]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    
    filters= [
    'areaType=region',
    'areaName=London'
    ]
    
    structure = {
    "date": "date",
    "areaName": "areaName",
    "areaCode": "areaCode",
    "vaccination_coverage": "cumVaccinationCompleteCoverageByVaccinationDatePercentage",
    "deaths": "cumDeaths28DaysByDeathDate"
    }
    
    
    api = Cov19API(filters=filters, structure=structure)
    
    data_new=api.get_json()
    return data_new # return data read from the API

In [25]:
def data_new_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        data_newdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
#data_newdf.plot()
#data_newdf.plot(logy=True)
        
series=wdg.SelectMultiple(
    options=['vaccination_coverage', 'deaths'],
    value=['vaccination_coverage'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#    value='pineapple', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Scale:',
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([series, scale])



# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
def refresh_graph():
    current=series.value
    series.value=[]
    series.value=current
    

graph=wdg.interactive_output(data_new_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)

Output()

In [24]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global data_newdf
    data_newdf=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh', # you may want to change this...
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to referesh data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='sync'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='success', description='Refresh', icon='sync', style=ButtonStyle(), tooltip='Click to refe…

**Author and Copyright Notice**  *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*